# DEMO: Empathy classification using a pattern classifier

In this notebook, it is possible to use a previously trained contrast-pattern classification algorithm to obtain the empathy level of a conversation between two people. 

In [92]:
import pickle
import pandas as pd
import torch
import os
import sys
import random 
import re
#import classifier
#from PBC4cip import PBC4cip
#from PBC4cip.core.Evaluation import obtainAUCMulticlass
#from PBC4cip.core.Helpers import get_col_dist, get_idx_val

#utilities for database management
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import os
import argparse

#import train_classifier as trainer
#import test_classifier as tester
#import database_processing_package as data_processer


First, we will obtain the model and sample conversation prompts

In [76]:
current_dir = os.getcwd() #get directory of the repository
database_dir = '/processed_databases/EmpatheticExchanges'
model_directory = current_dir + '/Experiments/outputs/Experiment '+ str(25) + '/' + 'trained_pbc4cip.sav'

print(model_directory)

database = pd.read_csv(current_dir + database_dir + '/EmpatheticExchanges.csv')

starting_exchange_db = database[database['exchange_number'] == 1]
starting_exchange_db = starting_exchange_db.reset_index()
len_of_db = len(starting_exchange_db)
index_of_sample = random.randint(0, len_of_db)
sample = starting_exchange_db.loc[index_of_sample,'speaker_utterance']
sample = re.sub("_comma_", ',', sample)
database 
sample


/home/haru/EmpathyClassification_ECPC/Experiments/outputs/Experiment 25/trained_pbc4cip.sav


'Man, I let one of my friends take my Benz one day to run some errands. I really thought she would be careful with it.'

In [85]:
flag = True
while(flag):
    answer = input("Provide your response: ")
    if answer.lower() == '':
        print('No answer received, please provide a response')
    else:
        flag = False

answer

Provide your response:  Oh no, why?


'Oh no, why?'

In [90]:
dataframe_columns = starting_exchange_db.columns
for i in dataframe_columns:
    print(i)

index
conv_id
context
prompt
speaker_utterance
listener_utterance
exchange_number
s_negative
s_neutral
s_positive
l_negative
l_neutral
l_positive
predictions_ER
predictions_IP
predictions_EX
s_word_len
l_word_len
agreeing
acknowledging
encouraging
consoling
sympathizing
suggesting
questioning
wishing
neutral
mimicry
empathy
